Tiedot perustuvat kirjan [Applied Predictive Modeling](https://link.springer.com/book/10.1007/978-1-4614-6849-3) kappaleeseen Data Pre-processing.

# Multikollineaarisuus

Multikollineaarisuus tarkoittaa tilannetta, jossa usean muuttujan välillä esiintyy vahvaa korrelaatiota; toisin sanoen nämä muuttujat kertovat saman asian, mutta jokainen hieman eri tavalla.

## Mitä haittaa

Tavoitteena on hyvän yleistyskyvyn omaava mahdollisimman yksinkertainen malli. Päällekkäistä tietoa sisältävät muuttujat lisäävät mallin monimutkaisuutta tuomatta lisäinformaatiota.

Oppaan kirjoittajat nostavat esille lineaarisen regression kaltaiset tekniikat, joilla multikollineaariset aineistot voivat heikentää mallin suorituskykyä ja luotettavuutta.

## Kuinka tunnistaa

Korrelaatiomatriisin visuaalinen tarkastelu voi paljastaa keskenään korreloivia ryhmiä. R-kielen [corrplot-funktion](https://cran.r-project.org/web/packages/corrplot/vignettes/corrplot-intro.html) avulla korrelaatiomatriisi voidaan tulostaa siten, että keskenään korreloivat muuttujat esitellään vierekkäin.

Multikollineaarisuutta voidaan selvittää myös [pääkomponenttianalyysin avulla](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) (PCA). Mikäli muutama pääkomponentti selittää suurimman osan aineiston varianssista, viittaa tämä siihen, että muuttujien välillä esiintyy multikollineaarisuutta. Komponenttien loadings arvojen tarkastelu voi auttaa tunnistamaan mistä muuttujista on kyse.

## Kuinka päästä eroon

Regressioanalyysiin on kehitetty erilaisia menetelmiä multikollineaarisuuden tunnistamiseen. 

Kun tietyn aineiston perustella on laadittu regressiomalli, niin VIF-mittarin avulla voi arvioida kärsiikö malli multikollineaarisuuten liittyvistä ongelmista ja pyrkiä tarvittaessa tunnistamaan ja poistamaan kollineaariset muuttujat.

VIF-mittarin avulla saatua tietoa ei kuitenkaan yleensä voi suoraan hyödyntää toisenlaisia malleja laadittaessa.

[Pääkomponenttianalyysi](https://fi.wikipedia.org/wiki/P%C3%A4%C3%A4komponenttianalyysi) tarjoaa tehokkaan menetelmän multikollineaarisuuden poistamiseen. 

PCA:n haittana on mallinen selitettävyyden vaikeutuminen. Kun alkuperäisistä muuttujista siirrytään pääkomponenttien käyttöön, kausaalisten yhteyksien selittäminen hankaloituu merkittävästi. Lisäksi, PCA tutkii ainoastaan selittävien muuttujien varianssia, eikä huomioi selittävien ja itsenäisen muuttujien välistä vuorovaikutusta. On kehitetty [PLS:n](https://en.wikipedia.org/wiki/Partial_least_squares_regression) kaltaisia menetelmiä, jotka paremmin huomioivat edellä mainitun yhteden.

Ladataan käytettävät paketit

In [1]:
import numpy as np
import pandas as pd

from copy import copy, deepcopy

In [3]:
def check_symmetric(a, rtol=1e-05, atol=1e-08):
    return np.allclose(a, a.T, rtol=rtol, atol=atol)

In [5]:
def findCorrelation(x, cutoff = 0.9, verbose = False):

    removed = []

    # Varmistetaan, että aineiston tietotyypit ovat float tyyppisiä
    if (x.dtypes.values[:, None] == ['int64', 'int32', 'int16', 'int8']).any():
            x = x.astype(float)

    # Irroitetaan arvot Numpy-taulukkoon
    _x = x.values

    # Napataan rivien lkm muistiin
    varnum = _x.shape[0]

    # Varmistetaan, että matriisi symmetrinen
    if check_symmetric(_x) == False:
        print("Ei oo symmetrinen")
        return None
    
    # Mitä tämä tarkoittaa?
    if varnum == 1:
        print("only one variable given")
        return None
    
    # Jatketaan lukujen absoluuttisilla arvoilla
    _x = np.abs(_x)


    # tyhjennetään diagonaaliakseli
    # - oletus on, että korrelaatiomatriisi sisältää float-arvoja....
    # - na.arvoja ei huomioda keskiarvoja laskettaessa
    np.fill_diagonal(_x, np.nan)

    # Muuttujien järjestys korrelaatioarvojen keskiarvon perustella
    maxAbsCorOrder = np.nanmean(_x, axis = 0).argsort()[::-1]

    for i in range(len(maxAbsCorOrder) - 1):
        iIndx = maxAbsCorOrder[i]

        # Ota jotenkin käyttöön...
        # if np.any(_x > cutoff) == False:
        #    print("Ei enää", i)
        # else:
        #    print(".", sum(_x > cutoff))

        # Onko sarake jo aiemmin poistettu
        if iIndx in removed:
            continue

        for j in range(i + 1, len(maxAbsCorOrder)):

            jIndx = maxAbsCorOrder[j]

            # Onko jompikumpi jo poistettu
            if iIndx in removed or jIndx in removed:
                continue
            
            # Ylittääkö muuttujien välinen korrelaatio raja-arvon
            if _x[iIndx][jIndx] > cutoff:
                    
                mn1 = np.nanmean(_x[iIndx,:])
                mn2 = np.nanmean(_x[:,jIndx])

                if mn1 > mn2:

                    removed.append(iIndx)
                    _x[iIndx,:] = np.nan
                    _x[:,iIndx] = np.nan
                else:

                    removed.append(jIndx)
                    _x[jIndx,:] = np.nan
                    _x[:,jIndx] = np.nan


    return removed


Määritellään testiaineisto:

In [9]:

np.random.seed([3,1415])
df = pd.DataFrame(
    np.random.randint(10, size=(10, 10)),
    columns=list('ABCDEFGHIJ'))

corr = df.corr()
hc = findCorrelation(corr, cutoff=0.5)

df.columns[hc]

Index(['H', 'J', 'G'], dtype='object')